In [49]:
!pip install torchattacks
!pip install pandas
!pip install openpyxl
!pip install numpy
!pip install plot 
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [50]:
import torch
import torch.nn as nn
import time
import pandas as pd

In [51]:
import Sources.DefaultModel as Util
import Sources.MnistModel as MnistModel
import Sources.TestUtils as TestUtils

In [52]:
Device = Util.HardwareIdentification()
TrainDataLoaderMNIST,TestDataLoaderMNIST = MnistModel.ImportData(1)
print(len(TrainDataLoaderMNIST.dataset), len(TestDataLoaderMNIST.dataset))

cuda
60000
10000
60000 10000


In [53]:
Model = MnistModel.ConvNeuralNet(10)
Model = Model.to(Device)
Epochs = 40
Loss = nn.CrossEntropyLoss()
LearningRate = 0.001
Optimizer = torch.optim.Adam(Model.parameters(), lr=LearningRate)

In [54]:
%%script false --no-raise-error
for Epoch in range(Epochs):
    ReturnLoss = Util.Trainloop(TrainDataLoaderMNIST, Model, Loss, Optimizer, Device)
    print('{:.4f}'.format(ReturnLoss))
ModelPath = 'Models/ModelMNIST.torch'
torch.save(Model.state_dict(), ModelPath)

In [55]:
Model.load_state_dict(torch.load('Models/ModelMNIST.torch'))

<All keys matched successfully>

In [56]:
Util.TestLoop(TestDataLoaderMNIST, Model, Device)
X, y = next(iter(TestDataLoaderMNIST))

Accuracy: 99.38%


In [57]:
ActivationList = [3, 4]
CoefList = [5]
CList = [10, 100]
ProbList = [4]
SimilarityList = [0.2]
Init = 0
lr = 0.2
Steps = 10000

data = []
for S in SimilarityList:
  for P in ProbList:
    for C in CList:
      for Coef in CoefList:
        for Activation in ActivationList:
          DataResult, _ = TestUtils.AttackNorms(Model, C, Steps, lr,  Coef, Activation, P, S, Init, X, y, Device)
          data += DataResult

df = pd.DataFrame(data, columns=['L0 Norm', 'L2 Norm', 'Linf Norm', 'Activation', 'Coef', 'Prob', 'c', 'Similarity', 'Pre', 'Ppred', 'Apred', 'Spre'])
df.to_excel("Results/Parameters/Parameters.xlsx")

20 2.4217021 0.9959918 3 5 10 4 0.2 7 7 2 7
18 3.2274702 0.99604154 4 5 10 4 0.2 7 7 2 7
27 2.8095863 0.994486 3 5 100 4 0.2 7 7 2 7
22 4.133838 0.9960615 4 5 100 4 0.2 7 7 3 7


In [58]:
Activation = 4
Coef = 5
Prob = 2
Similarity = 0.2
Init=1
lr = 0.02
Steps = 10000


data = []
start_time = time.time()
data = TestUtils.TestLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results/MNIST/ComputationTimeResults.txt")
TestUtils.SaveData(data, "Results/MNIST/Results.xlsx")

32 4.88465 0.9836782 4 5 1000 2 0.2 7 7 3 7
25 4.4969134 0.9939383 4 5 666.6666666666666 2 0.2 7 7 9 7
40 4.335822 0.9633353 4 5 444.4444444444444 2 0.2 7 7 2 7
22 4.2250414 0.99566245 4 5 296.29629629629625 2 0.2 7 7 1 7
17 3.834635 0.9964514 4 5 197.53086419753083 2 0.2 7 7 8 7
21 3.947553 0.99381864 4 5 131.68724279835388 2 0.2 7 7 1 7
13 3.409771 0.99925196 4 5 87.79149519890258 2 0.2 7 7 9 7
19 3.539122 0.99852556 4 5 58.527663465935056 2 0.2 7 7 1 7
9 2.6828792 0.98189956 4 5 39.01844231062337 2 0.2 7 7 2 7
23 3.6430566 0.9964371 4 5 26.012294873748914 2 0.2 7 7 9 7
86 3.030816 0.78963935 4 5 17.34152991583261 2 0.2 7 7 1 7
12 2.964212 0.99834144 4 5 11.561019943888406 2 0.2 7 7 2 7
15 3.2628496 0.995721 4 5 7.707346629258937 2 0.2 7 7 2 7
8 2.4544992 0.99756145 4 5 5.138231086172625 2 0.2 7 7 2 7
7 2.3990915 0.9993845 4 5 3.42548739078175 2 0.2 7 7 2 7
706 5.896355 0.4473731 4 5 2.283658260521167 2 0.2 7 7 2 7
706 6.2262874 0.48690236 4 5 1.5224388403474445 2 0.2 7 7 8 7
704 5.7

In [59]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=1
lr = 0.02

TestedIterations = [100, 1000]

for i in range((len(TestedIterations))):
    data = []
    start_time = time.time()
    data = TestUtils.TestLoopC(Model, TestedIterations[i], lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
    end_time = time.time()

    TestUtils.SaveTime(start_time, end_time, "Results/MNIST/TimeXM" + str(TestedIterations[i]) +".txt")
    TestUtils.SaveData(data, "Results/MNIST/ResultXM" + str(TestedIterations[i]) + ".xlsx")
    

763 13.529208 0.6322329 4 5 1000 2 0.2 7 7 1 7
755 13.45069 0.6447298 4 5 666.6666666666666 2 0.2 7 7 1 7
758 13.121813 0.6627497 4 5 444.4444444444444 2 0.2 7 7 1 7
744 12.114229 0.71204937 4 5 296.29629629629625 2 0.2 7 7 1 7
730 11.179966 0.68731594 4 5 197.53086419753083 2 0.2 7 7 1 7
648 9.812208 0.6799894 4 5 131.68724279835388 2 0.2 7 7 1 7
575 8.435994 0.74999034 4 5 87.79149519890258 2 0.2 7 7 1 7
445 6.8470664 0.6789295 4 5 58.527663465935056 2 0.2 7 7 1 7
239 4.905822 0.7070337 4 5 39.01844231062337 2 0.2 7 7 8 7
272 4.965718 0.5450983 4 5 26.012294873748914 2 0.2 7 7 1 7
232 4.906239 0.70526004 4 5 17.34152991583261 2 0.2 7 7 4 7
254 4.6250153 0.5961207 4 5 11.561019943888406 2 0.2 7 7 1 7
74 3.1214347 0.74433553 4 5 7.707346629258937 2 0.2 7 7 9 7
510 5.561542 0.61952513 4 5 5.138231086172625 2 0.2 7 7 8 7
563 5.5930467 0.5748867 4 5 3.42548739078175 2 0.2 7 7 8 7
708 5.8278885 0.41885537 4 5 2.283658260521167 2 0.2 7 7 2 7
703 5.836741 0.38042212 4 5 1.5224388403474445 2 

In [60]:
TestUtils.SaveImages("Results/MNIST/InitialImages/", X, y)

In [61]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=1
lr = 0.02
Steps = 10000

TestUtils.SaveLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5, "Results/MNIST/Images/")

38 4.935331 0.96344393 4 5 1000 2 0.2 7 7 8 7
23 3.970373 0.9639157 4 5 666.6666666666666 2 0.2 7 7 2 7
27 4.325328 0.9935932 4 5 444.4444444444444 2 0.2 7 7 8 7
30 4.5365605 0.98776686 4 5 296.29629629629625 2 0.2 7 7 1 7
19 3.908887 0.9921688 4 5 197.53086419753083 2 0.2 7 7 2 7
27 3.8938026 0.986916 4 5 131.68724279835388 2 0.2 7 7 8 7
15 3.247142 0.99358124 4 5 87.79149519890258 2 0.2 7 7 2 7
16 3.174768 0.98426914 4 5 58.527663465935056 2 0.2 7 7 9 7
16 3.6187434 0.99935055 4 5 39.01844231062337 2 0.2 7 7 2 7
21 3.445422 0.9993062 4 5 26.012294873748914 2 0.2 7 7 9 7
10 2.8755581 0.9999648 4 5 17.34152991583261 2 0.2 7 7 2 7
13 2.920763 0.9942975 4 5 11.561019943888406 2 0.2 7 7 2 7
9 2.902559 0.9903768 4 5 7.707346629258937 2 0.2 7 7 9 7
9 2.429898 0.9835472 4 5 5.138231086172625 2 0.2 7 7 2 7
5 2.0726795 0.97225964 4 5 3.42548739078175 2 0.2 7 7 2 7
708 6.547648 0.4364003 4 5 2.283658260521167 2 0.2 7 7 8 7
9 2.437407 0.99927425 4 5 1.5224388403474445 2 0.2 7 7 2 7
706 6.305149 

In [62]:
dataJSMA = []

start_time = time.time()
Data = TestUtils.TestJSMA(1,1, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results/MNIST/TimeJSMAM.txt")
TestUtils.SaveData(Data, "Results/MNIST/BestJSMAM.xlsx")

66 7.831178 1.0 0 0 0 0 0 7 7 8 7
[[66, array(7.831178, dtype=float32), array(1., dtype=float32), 0, 0, 0, 0, 0, 7, 7, 8, 7]]


In [64]:
TestUtils.SaveJSMA(1,1, Model, X, y, Device, "Results/MNIST/JSMAImages/")

66 7.831178 1.0 0 0 0 0 0 7 7 8 7


[[]]

In [63]:
Count=0
dataCW = []

start_time = time.time()
Data, AX = TestUtils.TestCW(1000, 1, 10, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results/MNIST/TimeCWM.txt")
TestUtils.SaveData([Data], "Results/MNIST/BestCWM.xlsx")

10
Attack iteration 0
Forced equal: tensor(0.) Equal count: tensor(603, device='cuda:0') tensor(784.) torch.Size([1, 1, 28, 28])
Forced equal: tensor(8.) Equal count: tensor(624, device='cuda:0') tensor(776.) torch.Size([1, 1, 28, 28])
Forced equal: tensor(16.) Equal count: tensor(626, device='cuda:0') tensor(768.) torch.Size([1, 1, 28, 28])
Forced equal: tensor(24.) Equal count: tensor(628, device='cuda:0') tensor(760.) torch.Size([1, 1, 28, 28])
Forced equal: tensor(32.) Equal count: tensor(627, device='cuda:0') tensor(752.) torch.Size([1, 1, 28, 28])
Forced equal: tensor(40.) Equal count: tensor(631, device='cuda:0') tensor(744.) torch.Size([1, 1, 28, 28])
Forced equal: tensor(48.) Equal count: tensor(631, device='cuda:0') tensor(736.) torch.Size([1, 1, 28, 28])
Forced equal: tensor(56.) Equal count: tensor(631, device='cuda:0') tensor(728.) torch.Size([1, 1, 28, 28])
Forced equal: tensor(64.) Equal count: tensor(631, device='cuda:0') tensor(720.) torch.Size([1, 1, 28, 28])
Forced e